In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_iris

iris = load_iris()
df = pd.DataFrame(data = iris.data, columns = iris.feature_names)

df.head(3)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2


### K-Means

In [3]:
from sklearn.cluster import KMeans

kmeans_3 = KMeans(n_clusters = 3, init = 'k-means++', max_iter = 15, random_state = 2045)

kmeans_3.fit(iris.data)

KMeans(max_iter=15, n_clusters=3, random_state=2045)

In [4]:
df['kmeans'] = kmeans_3.labels_
df['target'] = iris.target

df.groupby('target')['kmeans'].value_counts()

target  kmeans
0       1         50
1       0         48
        2          2
2       2         36
        0         14
Name: kmeans, dtype: int64

### Mean Shift(평균 이동)

- 데이터의 분포를 이동하여 군집의 중심을 탐색
  - 데이터 밀도가 가장 높은 곳으로 중심을 지속적으로 이동
- 군집의 중심점은 data points가 모여있는 곳이라는 가정
  - 확률 밀도 함수(Probability Density Function)
  - KDE(Kernel Density Estimation)

In [5]:
from sklearn.cluster import MeanShift

meanshift = MeanShift(bandwidth = 0.86)

meanshift.fit_predict(iris.data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0], dtype=int64)

In [6]:
from sklearn.cluster import estimate_bandwidth

estimate_bandwidth(iris.data) # recommendation

1.2020768127998687

In [7]:
df['meanshift'] = meanshift.fit_predict(iris.data)

df.groupby('target')['meanshift'].value_counts()

target  meanshift
0       1            50
1       0            48
        1             1
        2             1
2       2            36
        0            14
Name: meanshift, dtype: int64

### GMM(Gaussian Mixture Model)
- 데이터가 여러 개의 가우시안 분포(Gaussian Distribution)를 가진 데이터들의 집합이라고 가정
  - 개별 정규분포의 모수(평균, 분산) 추정
  - 각 data points가 어떤 정규분포에 해당하는지 확률 추정
    - EM(Expectation and Maximization)

In [8]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components = 3, random_state = 2045)

gmm.fit(iris.data)

GaussianMixture(n_components=3, random_state=2045)

In [9]:
gmm.predict(iris.data)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [10]:
df['gmm'] = gmm.predict(iris.data)

df.groupby('target')['gmm'].value_counts()

target  gmm
0       1      50
1       0      45
        2       5
2       2      50
Name: gmm, dtype: int64

### DBSCAN(Density Based Spatial Clustering of Applications with Noise)

- 밀도(Density) 기반 군집
  - 기하학적으로 복잡한 데이터에도 효과적으로 군집 가능
  - 핵심 포인트(Core Point)들을 서로 연결하면서 군집화

- Hyperparameter
  - epsilon(esp, 입실론 주변 영역)
    - 개별 data points를 중심으로 '입실론 반경'을 가지는 주변 영역
    - 'Core Point' 기준
  - min point(min_samples, 최소 데이터 개수)
    - 개별 data points의 '입실론 주변 영역'에 포함되는 다른 data points의 개수
    - 조건 만족 시 'Core Point'로 지정

In [11]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps = 0.8, min_samples = 8, metric = 'euclidean')

dbscan.fit_predict(iris.data)

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
      dtype=int64)

In [12]:
df['dbscan'] = dbscan.fit_predict(iris.data)
df.groupby('target')['dbscan'].value_counts()

target  dbscan
0        0        50
1        1        50
2        1        47
        -1         3
Name: dbscan, dtype: int64